In [42]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

In [43]:
df = pd.read_csv('variables.csv', index_col=0)
df

,CO(mg/m^3),Volume(m^3),N_people,Ambient-Air-Pump(L/min),Ambient-Air-Pump_power(%),Ambient-Air-Pump_number,CO(mg/m^3)_Dt
0,5.655523,20.705141,0.000000,418.161628,29.781933,3,5.350641
1,7.370026,28.080885,5.872497,333.977500,63.226408,3,7.721080
2,6.211477,20.047931,0.000000,608.093630,86.656211,8,5.113339
3,5.615021,12.017400,9.508670,698.808887,86.251549,8,5.135903
4,4.365763,20.551640,3.429715,562.049570,20.594074,4,4.977466
...,...,...,...,...,...,...,...
19995,0.026110,14.521764,0.000000,504.581243,68.026447,4,0.063048
19996,0.061070,21.680377,0.000000,306.288852,11.767788,6,0.098633
19997,0.099518,26.692921,0.000000,257.708970,32.729522,7,0.077012
19998,0.066477,11.676167,0.000000,533.694343,0.000000,8,0.097035


In [44]:
df_X = df[df.columns[:-1]]
df_X

,CO(mg/m^3),Volume(m^3),N_people,Ambient-Air-Pump(L/min),Ambient-Air-Pump_power(%),Ambient-Air-Pump_number
0,5.655523,20.705141,0.000000,418.161628,29.781933,3
1,7.370026,28.080885,5.872497,333.977500,63.226408,3
2,6.211477,20.047931,0.000000,608.093630,86.656211,8
3,5.615021,12.017400,9.508670,698.808887,86.251549,8
4,4.365763,20.551640,3.429715,562.049570,20.594074,4
...,...,...,...,...,...,...
19995,0.026110,14.521764,0.000000,504.581243,68.026447,4
19996,0.061070,21.680377,0.000000,306.288852,11.767788,6
19997,0.099518,26.692921,0.000000,257.708970,32.729522,7
19998,0.066477,11.676167,0.000000,533.694343,0.000000,8


In [45]:
df_y = df[df.columns[-1:]]
df_y

,CO(mg/m^3)_Dt
0,5.350641
1,7.721080
2,5.113339
3,5.135903
4,4.977466
...,...
19995,0.063048
19996,0.098633
19997,0.077012
19998,0.097035


In [46]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=True, random_state=42)

In [47]:
# try linear regression with sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.metrics import mean_absolute_error
reg = LinearRegression().fit(X_train, y_train)
pred = reg.predict(X_test)
true = y_test.values
mean_absolute_error(true, pred)

0.35118297390981523

In [48]:
# add prefix to column names
y_pred = pd.DataFrame(pred, columns="pred_" + y_test.columns, index=y_test.index)
y_test.columns = 'true_' + y_test.columns
# compare predictions with actual values
y_compare = pd.concat([y_test, y_pred], axis=1)
y_compare

,true_CO(mg/m^3)_Dt,pred_CO(mg/m^3)_Dt
71787,9.728845,9.833978
67218,0.904454,0.797386
54066,5.006690,4.830161
7168,2.319558,3.361563
29618,5.024888,5.235538
...,...,...
17283,15.164946,11.837295
89286,5.095898,5.539426
47263,2.887302,2.459969
83913,9.470092,9.133036


In [49]:
# plot predictions
fig = px.line(y_compare.iloc[:100], x= [i for i in range(0,100)], y = ["true_CO(mg/m^3)_Dt", "pred_CO(mg/m^3)_Dt"], title='Predictions vs true values')
fig.show()

In [50]:
# retrain on the entire dataset
reg = LinearRegression().fit(df_X, df_y)
with open('linear_regressor_predictor.pkl', 'wb') as f:
    pickle.dump(reg, f)